In [29]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from sklearn import linear_model

In [78]:
df = pd.read_excel('table_8_offenses_known_to_law_enforcement_new_york_by_city_2013.xls')


In [79]:
df = pd.DataFrame(df.values[4:], columns=df.iloc[3])

In [80]:
# drop last 3 rows
df.drop(df.tail(3).index,inplace=True)

In [81]:
# keep only necessary columns
df_final = df[['Population', 'Property\ncrime']]
df_final['murder'] = df['Murder and\nnonnegligent\nmanslaughter'].apply(lambda x : 1 if x>0 else 0)
df_final['robbery'] = df['Robbery'].apply(lambda x : 1 if x>0 else 0)

C:\Users\julia\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\julia\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [87]:
df_final['Pop_square'] = df['Population'] **2
df_final['Violent\ncrime'] = df['Violent\ncrime'] 
df_final['Murder'] = df['Murder and\nnonnegligent\nmanslaughter'] 
df_final['Robbery'] = df['Robbery'] 
df_final['Aggravated\nassault'] = df['Aggravated\nassault'] 
df_final['Burglary'] = df['Burglary'] 
df_final['Motor\nvehicle\ntheft'] = df['Motor\nvehicle\ntheft'] 
#df_final['Arson3'] = df['Arson3'] 

C:\Users\julia\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\julia\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\julia\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.

In [83]:
df_final.dropna(inplace=True)

C:\Users\julia\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [88]:
len(df_final)

348

In [91]:
regr = linear_model.LinearRegression()
y = df_final['Property\ncrime']
X = df_final.drop('Property\ncrime',axis=1)


In [90]:
#df.drop('City',axis=1,inplace=True)
#df.drop('Rape\n(legacy\ndefinition)2',axis=1,inplace=True)
#df.drop('Rape\n(revised\ndefinition)1',axis=1,inplace=True)
df_final.head()

3,Population,Property crime,murder,robbery,Pop_square,Violent crime,Murder,Robbery,Aggravated assault,Burglary,Motor vehicle theft
0,1861,12,0,0,3463321,0,0,0,0,2,0
1,2577,24,0,0,6640929,3,0,0,3,3,1
2,2846,16,0,0,8099716,3,0,0,3,1,0
3,97956,4090,1,1,9595377936,791,8,227,526,705,142
4,6388,223,0,1,40806544,23,0,4,16,53,5


In [92]:
regr.fit(X, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [93]:
print('\nCoefficients: \n', regr.coef_)
print('\nIntercept: \n', regr.intercept_)
print('\nR-squared:')
print(regr.score(X, y))


Coefficients: 
 [ 1.07155049e-02 -1.89291993e+01  3.56401136e+01  4.41644437e-10
  2.06261378e+00 -8.97060454e-01 -3.45009837e+00 -2.21842756e+00
  3.81849144e+00 -1.83807308e+00]

Intercept: 
 -29.948228012091363

R-squared:
0.9996469753778969


In [107]:
from sklearn.model_selection import cross_val_score
print(cross_val_score(regr, X, y ,cv=10))

[  0.90272005   0.9901784    0.85344787   0.91916564   0.89424046
   0.92984316 -68.21757784   0.9874555    0.92608538   0.81556884]


In [112]:
from sklearn.decomposition import PCA 
sklearn_pca = PCA(n_components=4)
X_pca = sklearn_pca.fit_transform(X)
print(cross_val_score(regr, X_pca, y ,cv=10))

[   0.92420068    0.99076364    0.85887566    0.92589078    0.89959994
    0.94889803 -133.44033153    0.99152408    0.9778909     0.85589393]


In [117]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2)
regr.fit(X_train, y_train)
print('\nCoefficients: \n', regr.coef_)
print('\nIntercept: \n', regr.intercept_)
print('\nR-squared:')
print(regr.score(X_test, y_test))


Coefficients: 
 [ 2.00881719e-09  3.39075709e-02  1.99670261e+00 -3.33229412e+00]

Intercept: 
 791.1448455487183

R-squared:
0.9416895674634239


In [119]:
print(cross_val_score(regr, X_test, y_test,cv=10))

[0.96350148 0.83846337 0.91518017 0.91552281 0.4208813  0.99418987
 0.93058107 0.97898332 0.9053675  0.97411796]
